In [1]:
import pandas as pd
import numpy as np
from math import pi, radians
from matplotlib import pyplot as plt
import tensorflow as tf

In [2]:
def normalize(vec):
    max_val = max(vec)
    min_val = min(vec)
    return (vec - min_val) / (max_val - min_val)

def stardandize(vec):
    mean = np.mean(vec)
    std = np.std(vec)
    return (vec - mean) / std

In [3]:
# Cartesian coordinates
trace_data = pd.read_csv("trace/trace_avg.csv")
trace_data_time = pd.to_datetime(trace_data['Time'])
long = np.radians(trace_data['Long'])
lat = np.radians(trace_data['Lat'])
trace_data['X'] = np.cos(lat) * np.cos(long)
trace_data['Y'] = np.cos(lat) * np.sin(long)
trace_data['Z'] = np.sin(lat)

In [4]:
fogs_data = pd.read_csv("trace/fogs.csv")

long = np.radians(fogs_data['Long'])
lat = np.radians(fogs_data['Lat'])
fogs_data['X fog'] = np.cos(lat) * np.cos(long)
fogs_data['Y fog'] = np.cos(lat) * np.sin(long)
fogs_data['Z fog'] = np.sin(lat)
fogs_data = fogs_data.drop(columns=['Long', 'Lat'])

trace_data = trace_data.join(fogs_data.set_index('Fog node'), on='Fog node')
trace_data['X rel'] = trace_data['X'] - trace_data['X fog']
trace_data['Y rel'] = trace_data['Y'] - trace_data['Y fog']
trace_data['Z rel'] = trace_data['Z'] - trace_data['Z fog']

lat_star = np.average(trace_data['Lat'])
long_star = np.average(trace_data['Long'])

trace_data['x'] = trace_data['X'] - np.cos(lat_star) * np.cos(long_star)
trace_data['y'] = trace_data['Y'] - np.cos(lat_star) * np.sin(long_star)
trace_data['z'] = trace_data['Z'] - np.sin(lat_star)

trace_data['x'] = - trace_data['x'] * np.sin(long_star) + trace_data['y'] * np.cos(long_star)
trace_data['y'] = - trace_data['x'] * np.cos(long_star) * np.sin(lat_star) - trace_data['y'] * np.sin(long_star) * np.sin(lat_star) + trace_data['z'] * np.cos(lat_star)
trace_data = trace_data.drop(columns=['z'])

trace_data['x fog'] = trace_data['X fog'] - np.cos(lat_star) * np.cos(long_star)
trace_data['y fog'] = trace_data['Y fog'] - np.cos(lat_star) * np.sin(long_star)
trace_data['z fog'] = trace_data['Z fog'] - np.sin(lat_star)

trace_data['x fog'] = - trace_data['x fog'] * np.sin(long_star) + trace_data['y fog'] * np.cos(long_star)
trace_data['y fog'] = - trace_data['x fog'] * np.cos(long_star) * np.sin(lat_star) - trace_data['y fog'] * np.sin(long_star) * np.sin(lat_star) + trace_data['z fog'] * np.cos(lat_star)
trace_data = trace_data.drop(columns=['z fog'])

trace_data['x rel'] = trace_data['x'] - trace_data['x fog']
trace_data['y rel'] = trace_data['y'] - trace_data['y fog']

trace_data['r'] = np.sqrt(trace_data['x rel']**2 + trace_data['y rel']**2)
trace_data['theta'] = np.arctan(trace_data['y rel'] / trace_data['x rel'])

In [5]:
h = trace_data_time.dt.hour
m = trace_data_time.dt.minute
s = trace_data_time.dt.second
day = trace_data_time.dt.day
month = trace_data_time.dt.month
T = 2*pi*(s+60*m+60*60*h)/(60*60*24)
trace_data['CosTime'] = np.cos(T)
trace_data['SinTime'] = np.sin(T)

In [6]:
trace_data['Cost'] = trace_data['Cost'] / max(trace_data['Cost'])
trace_data['r'] = stardandize(trace_data['r'])

X_avg = np.average(trace_data['X'])
Y_avg = np.average(trace_data['Y'])
Z_avg = np.average(trace_data['Z'])

X_range = max(trace_data['X']) - min(trace_data['X'])
Y_range = max(trace_data['Y']) - min(trace_data['Y'])
Z_range = max(trace_data['Z']) - min(trace_data['Z'])

trace_data['X'] = (trace_data['X'] - X_avg) / X_range
trace_data['Y'] = (trace_data['Y'] - Y_avg) / Y_range
trace_data['Z'] = (trace_data['Z'] - Z_avg) / Z_range

trace_data['X fog'] = (trace_data['X fog'] - X_avg) / X_range
trace_data['Y fog'] = (trace_data['Y fog'] - Y_avg) / Y_range
trace_data['Z fog'] = (trace_data['Z fog'] - Z_avg) / Z_range

trace_data['X rel'] = trace_data['X rel'] / X_range
trace_data['Y rel'] = trace_data['Y rel'] / Y_range
trace_data['Z rel'] = trace_data['Z rel'] / Z_range

trace_data['Long'] = normalize(trace_data['Long'])
trace_data['Lat'] = normalize(trace_data['Lat'])

# trace_data['x'] = normalize(trace_data['x'])
# trace_data['y'] = normalize(trace_data['y'])

In [7]:
# Save preprocessed data
trace_data.to_csv("trace/trace_preprocessed.csv")